In [1]:
import argparse
import json
import os
import pandas as pd
import os

import datetime
import numpy as np
from cgms_data_seg_t1dexi import CGMSDataSeg
from cnn_ohio import regressor, regressor_transfer, test_ckpt
from data_reader_T1DEXI import DataReader

In [2]:
import tensorflow as tf

# New method in TensorFlow 2.x:
# This will list the devices TensorFlow recognizes
print("TensorFlow version:", tf.__version__)
print("List of devices available to TensorFlow:")
print(tf.config.list_physical_devices())

TensorFlow version: 2.10.1
List of devices available to TensorFlow:
[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [3]:
def read_preprocess_cgm_data(subj):
    subject = pd.read_csv(f'C:/Users/username/OneDrive/Desktop/BGprediction/LB_split/{subj}.csv')
    subject_cgm = subject.loc[subject['LBCAT'] == 'CGM']
    # Convert the 'LBDTC' column to datetime
    subject_cgm['LBDTC'] = pd.to_datetime(subject_cgm['LBDTC'])
    # Extract the date part
    subject_cgm['date'] = subject_cgm['LBDTC'].dt.date
    return subject_cgm
    # Count the number of occurrences of each unique date

In [4]:
overlap = ['854',
 '979',
 '816',
 '953',
 '981',
 '1617',
 '1343',
 '987',
 '255',
 '907',
 '856',
 '354',
 '894',
 '862',
 '900',
 '695']

In [ ]:
# Divide training and testing
# Generate folder for training and test data

for subj in overlap:
    subject = pd.read_csv(f'C:/Users/username/OneDrive/Desktop/BGprediction/LB_split/{subj}.csv')
    subject_cgm = subject.loc[subject['LBCAT'] == 'CGM']
    
    split_index = int(len(subject_cgm) * 0.8)
    # Split the DataFrame
    train_df = subject[:split_index]
    test_df = subject[split_index:]

    # Save the DataFrames to CSV files
    train_df.to_csv(f'C:/Users/username/OneDrive/Desktop/BGprediction/LB_split/training/{subj}_training_data.csv', index=False)
    test_df.to_csv(f'C:/Users/username/OneDrive/Desktop/BGprediction/LB_split/testing/{subj}_testing_data.csv', index=False)

    # Optionally, confirm the operation
    print(f"Training data saved with {len(train_df)} records.")
    print(f"Testing data saved with {len(test_df)} records.")

In [8]:
def read_preprocess_cgm_training(subj):
    subject_cgm = pd.read_csv(f'C:/Users/username/OneDrive/Desktop/BGprediction/LB_split/training/{subj}_training_data.csv')

    subject_cgm['LBDTC'] = pd.to_datetime(subject_cgm['LBDTC'])
    # Extract the date part
    subject_cgm['date'] = subject_cgm['LBDTC'].dt.date
    subject_cgm_concise = subject_cgm[["LBORRES", "LBDTC"]]
    return subject_cgm_concise
    

In [9]:
# An example 
subject_cgm = read_preprocess_cgm_training(overlap[1])

In [31]:
import datetime
# Segment the data into several pieces
# Assuming self.interval_timedelta is set, for example:
interval_timedelta = datetime.timedelta(minutes=6)  # Example timedelta of 6 minutes, providing a range for latency

# Create a list to store the results
res = []

# Initialize the first group
if not subject_cgm.empty:
    current_group = [subject_cgm.iloc[0]['LBORRES']]
    last_time = subject_cgm.iloc[0]['LBDTC']

# Iterate over rows in DataFrame starting from the second row
for index, row in subject_cgm.iloc[1:].iterrows():
    current_time = row['LBDTC']
    if (current_time - last_time) <= interval_timedelta:
        # If the time difference is within the limit, add to the current group
        current_group.append(row['LBORRES'])
    else:
        # Otherwise, start a new group
        res.append(current_group)
        current_group = [row['LBORRES']]
    last_time = current_time

# Add the last group if it's not empty
if current_group:
    res.append(current_group)

# Functions

In [5]:
def read_preprocess_cgm_training(subj):
    subject_cgm = pd.read_csv(subj)

    subject_cgm['LBDTC'] = pd.to_datetime(subject_cgm['LBDTC'])
    # Extract the date part
    subject_cgm['date'] = subject_cgm['LBDTC'].dt.date
    subject_cgm_concise = subject_cgm[["LBORRES", "LBDTC"]]
    return subject_cgm_concise
    

In [6]:
def preporcess_T1DEXI(subject_id):
    subject_cgm = read_preprocess_cgm_training(subject_id)
    interval_timedelta = datetime.timedelta(minutes=6)  # Example timedelta of 6 minutes, providing a range for latency

    # Create a list to store the results
    res = []

    # Initialize the first group
    if not subject_cgm.empty:
        current_group = [subject_cgm.iloc[0]['LBORRES']]
        last_time = subject_cgm.iloc[0]['LBDTC']

    # Iterate over rows in DataFrame starting from the second row
    for index, row in subject_cgm.iloc[1:].iterrows():
        current_time = row['LBDTC']
        if (current_time - last_time) <= interval_timedelta:
            # If the time difference is within the limit, add to the current group
            current_group.append(row['LBORRES'])
        else:
            # Otherwise, start a new group
            res.append(current_group)
            current_group = [row['LBORRES']]
        last_time = current_time

    # Add the last group if it's not empty
    if current_group:
        res.append(current_group)

    return res

# For loop to generate res for train and test

In [ ]:
# Define the directory path
train_directory_path = r'C:\Users\username\OneDrive\Desktop\BGprediction\LB_split\training'  # Use a raw string for paths on Windows

# List files without their extensions
train_file_names = [os.path.splitext(file)[0] for file in os.listdir(train_directory_path)
              if os.path.isfile(os.path.join(train_directory_path, file))]

# Print the list of file names
print(train_file_names)

In [12]:
# Define the directory path
test_directory_path = r'C:\Users\username\OneDrive\Desktop\BGprediction\LB_split\testing'  # Use a raw string for paths on Windows

# List files without their extensions
test_file_names = [os.path.splitext(file)[0] for file in os.listdir(test_directory_path)
              if os.path.isfile(os.path.join(test_directory_path, file))]

# Print the list of file names
print(test_file_names)

['1343_testing_data', '1617_testing_data', '255_testing_data', '354_testing_data', '695_testing_data', '816_testing_data', '854_testing_data', '856_testing_data', '862_testing_data', '894_testing_data', '900_testing_data', '907_testing_data', '953_testing_data', '979_testing_data', '981_testing_data', '987_testing_data']


In [13]:
train_data = dict()
for subj in train_file_names:
    print(subj)
    subj_path = f'C:/Users/username/OneDrive/Desktop/BGprediction/LB_split/training/{subj}.csv'
    reader = preporcess_T1DEXI(subj_path)
    train_data[subj] = reader

1343_training_data
1617_training_data
255_training_data
354_training_data
695_training_data
816_training_data
854_training_data
856_training_data
862_training_data
894_training_data
900_training_data
907_training_data
953_training_data
979_training_data
981_training_data
987_training_data


In [14]:
# Have not been run
test_data = dict()
for subj in test_file_names:
    print(subj)
    subj_path = f'C:/Users/username/OneDrive/Desktop/BGprediction/LB_split/testing/{subj}.csv'
    reader = preporcess_T1DEXI(subj_path)
    test_data[subj] = reader

1343_testing_data
1617_testing_data
255_testing_data
354_testing_data
695_testing_data
816_testing_data
854_testing_data
856_testing_data
862_testing_data
894_testing_data
900_testing_data
907_testing_data
953_testing_data
979_testing_data
981_testing_data
987_testing_data


In [15]:
epoch = 20

ph = 6 # Prediction horizon
path = "../t1dexi_results"

In [16]:
cleaned_subjects = [s.replace("_training_data", "") for s in train_file_names]
cleaned_subjects.sort()

In [19]:
# a dumb dataset instance
train_dataset = CGMSDataSeg(
    "ohio", "C:/Users/username/OneDrive/Desktop/BGprediction/LB_split/training/255_training_data.csv", 5
)
sampling_horizon = 7
prediction_horizon = ph
scale = 0.01
outtype = "Same"
# train on training dataset
# k_size, nblock, nn_size, nn_layer, learning_rate, batch_size, epoch, beta
with open(f'../t1dexi_results/config.json') as json_file:
    config = json.load(json_file)
argv = (
    config["k_size"],
    config["nblock"],
    config["nn_size"],
    config["nn_layer"],
    config["learning_rate"],
    config["batch_size"],
    epoch,
    config["beta"],
)
l_type = config["loss"]
# test on patients data
outdir = os.path.join(path, f"ph_{prediction_horizon}_{l_type}")
if not os.path.exists(outdir):
    os.makedirs(outdir)
all_errs = []

2020-04-01 00:01:51
Reading 8 segments


In [ ]:
# Loop
standard = False  # do not use standard
all_errs = []
for pid in overlap:
    # local_train_data = []
    
    # for k in train_data.keys():
    #     if k != pid:
    #         local_train_data += train_data[k]
    train_pids = set(overlap) - set([pid])
    local_train_data = []
    for k in train_pids:
        local_train_data += train_data[k + "_training_data"]
    print(f"Pretrain data: {sum([sum(x) for x in local_train_data])}")

    train_dataset.data = local_train_data
    train_dataset.set_cutpoint = -1
    train_dataset.reset(
        sampling_horizon,
        prediction_horizon,
        scale,
        100,
        False,
        outtype,
        1,
        standard,
    )
    regressor(train_dataset, *argv, l_type, outdir)
    # Fine-tune and test
    # target_test_dataset = CGMSDataSeg(
    #     "ohio", f"C:/Users/username/OneDrive/Desktop/BGprediction/OhioT1DM/2018/test/{pid}-ws-testing.xml", 5
    # )
    target_test_dataset = CGMSDataSeg(
    "ohio", f"C:/Users/username/OneDrive/Desktop/BGprediction/LB_split/testing/{pid}_testing_data.csv", 5
    )
    target_test_dataset.set_cutpoint = 1
    target_test_dataset.reset(
        sampling_horizon,
        prediction_horizon,
        scale,
        0.01,
        False,
        outtype,
        1,
        standard,
    )
    # target_train_dataset = CGMSDataSeg(
    #     "ohio", f"C:/Users/username/OneDrive/Desktop/BGprediction/OhioT1DM/2018/test/{pid}-ws-testing.xml", 5
    # )
    target_train_dataset = CGMSDataSeg(
    "ohio", f"C:/Users/username/OneDrive/Desktop/BGprediction/LB_split/training/{pid}_training_data.csv", 5
    )
    target_train_dataset.set_cutpoint = -1
    target_train_dataset.reset(
        sampling_horizon,
        prediction_horizon,
        scale,
        100,
        False,
        outtype,
        1,
        standard,
    )
    err, labels = test_ckpt(target_test_dataset, outdir)
    errs = [err]
    transfer_res = [labels]
    for i in range(1, 2):
        err, labels = regressor_transfer(
            target_train_dataset,
            target_test_dataset,
            config["batch_size"],
            epoch,
            outdir,
            i,
        )
        errs.append(err)
        transfer_res.append(labels)
    transfer_res = np.concatenate(transfer_res, axis=1)
    print(transfer_res)
    np.savetxt(
        f"{outdir}/{pid}.txt",
        transfer_res,
        fmt="%.4f %.4f %.4f %.4f", #%.4f %.4f %.4f %.4f
    )
    all_errs.append([pid] + errs)
all_errs = np.array(all_errs)
np.savetxt(f"{outdir}/errors.txt", all_errs, fmt="%s %.4f %.4f") # %.4f %.4f
# label pair:(groundTruth, y_pred)


In [21]:
all_errs

array([['854', '0.15074661', '0.14032947'],
       ['979', '0.10964842', '0.1068475'],
       ['816', '0.17296007', '0.15637168'],
       ['953', '0.16875254', '0.16648282'],
       ['981', '0.18782552', '0.1781756'],
       ['1617', '0.18750052', '0.17405139'],
       ['1343', '0.1479887', '0.12966709'],
       ['987', '0.1956755', '0.18187545'],
       ['255', '0.112035595', '0.11363354'],
       ['907', '0.2046883', '0.18899176'],
       ['856', '0.16526204', '0.15994464'],
       ['354', '0.21569382', '0.21845779'],
       ['894', '0.1430959', '0.14199'],
       ['862', '0.2226933', '0.19469088'],
       ['900', '0.14939056', '0.12725726'],
       ['695', '0.16643564', '0.16699977']], dtype='<U32')

In [22]:

# Convert the second and third columns to floats
second_column = all_errs[:, 1].astype(float)
third_column = all_errs[:, 2].astype(float)

# Calculate the average
average_second_column = np.mean(second_column)
average_third_column = np.mean(third_column)

print("Average of the second column:", average_second_column)
print("Average of the third column:", average_third_column)

Average of the second column: 0.16877456468750002
Average of the third column: 0.159110415


 The altimate result please go into the result folder to check the evaluation.ipynb

In [ ]:
# If run into an error, but all result txt files are ready, please run this section for evaluation
import argparse
import json
import os
import pandas as pd
import os
import datetime
import numpy as np

# List all files and directories in the current directory
files_and_directories = os.listdir('./t1dexi_results')

# Filter for files that end with .txt
txt_files = [file for file in files_and_directories if file.endswith('.txt')]

# Read the data from the text file
def calcuate_rmse(file):
    data = np.loadtxt(file)  # Make sure to replace 'data.txt' with your actual file path
    print(file)
    # Splitting the data into groundtruth and predictions
    groundtruth = data[:, 0]  # First column as ground truth (also same as third column)
    predictions_1 = data[:, 1]  # Second column as predictions from method 1
    predictions_2 = data[:, 3]  # Fourth column as predictions from method 2

    # Function to calculate RMSE
    def calculate_rmse(true_values, predictions):
        mse = np.mean((true_values - predictions) ** 2)
        rmse = np.sqrt(mse)
        return rmse

    # Calculate RMSE for each method
    rmse_method_1 = calculate_rmse(groundtruth, predictions_1)
    rmse_method_2 = calculate_rmse(groundtruth, predictions_2)

    print("RMSE for Method 1:", rmse_method_1)
    print("RMSE for Method 2:", rmse_method_2)
    return rmse_method_1


rmse_list = []
for f in txt_files[:-1]:
    rmse1 = calcuate_rmse(f)
    print(rmse1)
    rmse_list.append(rmse1)

print(np.average(rmse_list))


In [ ]:
rmse_list